In [139]:
import random
import glob
import os

from PIL import Image, ImageDraw, ImageFont, ImageFilter, ImageChops
def generate_calendar(text_items,image_path, output_path):
    # img = Image.new('RGB', (800, 800), 'black')
    img_ori = Image.open(image_path)
    w, h = img_ori.size
    min_side = min(w, h)
    left = (w - min_side) // 2
    top = (h - min_side) // 2
    right = left + min_side
    bottom = top + min_side
    img_cropped = img_ori.crop((left, top, right, bottom))
    img_resized = img_cropped.resize((800, 800), Image.LANCZOS)
    img_blurred = img_resized.filter(ImageFilter.GaussianBlur(radius=20))

    # 检查图像与白色是否具有足够的对比度，如果没有，则加上半透明黑色蒙版直到对比度足够
    def get_contrast(img):
        # 计算图像与白色的对比度（亮度均值与255的差值）
        grayscale = img.convert("L")
        avg_luminance = sum(grayscale.getdata()) / (img.width * img.height)
        return abs(255 - avg_luminance)

    contrast_threshold = 80  # 可根据实际需求调整
    contrast = get_contrast(img_blurred)
    while contrast < contrast_threshold:
        # 添加半透明黑色蒙版
        overlay = Image.new("RGBA", img_blurred.size, (0, 0, 0, 10))
        img_blurred = Image.alpha_composite(img_blurred.convert("RGBA"), overlay).convert("RGB")
        contrast = get_contrast(img_blurred)

    # 添加边框
    border_size = 10
    img_with_border = Image.new('RGB', (img_blurred.width + border_size * 2, img_blurred.height + border_size * 2), 'white')
    img_with_border.paste(img_blurred, (border_size, border_size))

    img = img_with_border
    draw = ImageDraw.Draw(img)
    # 文字阴影效果参数
    shadow_offset = 2
    shadow_color = "rgba(0,0,0,150)"  # 半透明黑色

    def draw_text_with_shadow(draw, position, text, font, fill, shadow_offset, shadow_color):
        x, y = position
        # 绘制阴影
        draw.text((x + shadow_offset, y + shadow_offset), text, font=font, fill=shadow_color)
        # 绘制文字
        draw.text((x, y), text, font=font, fill=fill)
    y = 0
    for font_size, text in text_items:
        text = text.strip() if text.strip() else " "  # 避免空字符串导致的问题
        font = ImageFont.truetype("SourceHanSerifCN-Heavy.otf", font_size)
        bbox = draw.textbbox((0, 0), text, font=font)
        w, h = bbox[2] - bbox[0], bbox[3] - bbox[1]
        if w > 720:
            # 如果宽度超过720，首先寻找标点符号进行自然换行，若过宽或过长，缩小字体直到合适
            # 标点符号列表
            punctuations = "，。！？；、,.!?;: "
            # 尝试在标点后换行
            wrapped = []
            line = ""
            for char in text:
                line += char
                if char in punctuations and len(line) > 8:
                    wrapped.append(line)
                    line = ""
            if line:
                wrapped.append(line)
            # 如果换行后仍有某行过宽，则缩小字体
            while True:
                too_wide = False
                for line in wrapped:
                    line = line.strip()
                    bbox_line = draw.textbbox((0, 0), line, font=font)
                    w_line = bbox_line[2] - bbox_line[0]
                    if w_line > 720:
                        too_wide = True
                        break
                if too_wide and font_size > 15:
                    font_size -= 2
                    font = ImageFont.truetype("SourceHanSerifCN-Heavy.otf", font_size)
                else:
                    break
            # 绘制每行
            for line in wrapped:
                line = line.strip()
                bbox_line = draw.textbbox((0, 0), line, font=font)
                w_line, h_line = bbox_line[2] - bbox_line[0], bbox_line[3] - bbox_line[1]
                # draw.text(((800-w_line)//2, y), line, fill="white", font=font)
                draw_text_with_shadow(draw, ((800 - w_line) // 2, y), line, font, "white", shadow_offset, shadow_color)
                y += h_line
            continue  # 当前文本已绘制，跳过后续 draw.text

        # draw.text(((800-w)//2, y), text, fill="white", font=font)
        draw_text_with_shadow(draw, ((800 - w) // 2, y), text, font, "white", shadow_offset, shadow_color)
        y += bbox[3]

    img.save(output_path)
    
def draw_calendar(data_root_path,output_root_path,year,month,day):
    random_seed = year * 10000 + month * 100 + day
    # 使用该随机数种子，获得data_root_path下的一个文件夹
    random.seed(random_seed)
    lyr_files = glob.glob(f"{data_root_path}/*/lyrs/*.txt")
    selected_lyr_file = random.choice(lyr_files)
    selected_folder = os.path.dirname(os.path.dirname(selected_lyr_file))

    # folders = glob.glob(f"{data_root_path}/*/")
    # selected_folder = random.choice(folders)
    # 在selected_folder/pics下随机选择一张图片
    pics_folder = os.path.join(selected_folder, "pics")
    pic_files = glob.glob(f"{pics_folder}/*")
    selected_pic = random.choice(pic_files)
    # # 在selected_folder/lyrs下随机选择一个txt文件中的随机一行
    # lyrs_folder = os.path.join(selected_folder, "lyrs")
    # lyr_files = glob.glob(f"{lyrs_folder}/*.txt")
    # selected_lyr_file = random.choice(lyr_files)
    with open(selected_lyr_file, "r", encoding="utf-8") as f:
        lyr_lines = f.readlines()
        # 保证选中的歌词行中有文字（非空且非仅空白字符）
        valid_lyr_lines = [line.strip() for line in lyr_lines if line.strip()]
        selected_lyr_line = random.choice(valid_lyr_lines)


    generate_calendar(
        # date="2023-10-05",
        # week_day="tuesday",
        # quote="门前的走出快来，后面的后面的跟牢靠来",
        text_items=[
            (1, " "),
            (360, f"{day:02d}"),
            (100, f"{year}-{month:02d}"),
            (50, " "),
            (36, selected_lyr_line),
        ],
        image_path=selected_pic,
        output_path=os.path.join(output_root_path, f"{year}-{month:02d}-{day:02d}.png"),
    )


In [ ]:
data_root_path_ = r"./example_root"
output_root_path_ = r"./output"

year_ = 2025
month_ = 8
day_ = 31

draw_calendar(data_root_path_, output_root_path_, year_, month_, day_)

In [142]:
data_root_path_ = r"./RCS"
output_root_path_ = r"./output"

year_ = 2025
for month_ in range(8, 13):
    output_month_path = os.path.join(output_root_path_, f"{year_}-{month_:02d}")
    os.makedirs(output_month_path, exist_ok=True)
    for day_ in range(1, 32):
        draw_calendar(data_root_path_, output_month_path, year_, month_, day_)